In [1]:
import sys
import os
import time
from pilot import PilotComputeService, ComputeDataService, ComputeDataServiceDecentral, State

COORDINATION_URL = "redis://localhost:6379"

/home/01131/tg804093/anaconda2/lib/python2.7/site-packages/radical/utils/atfork/stdlib_fixer.py:58: UserWarning: logging module already imported before fixup.
  warnings.warn('logging module already imported before fixup.')
/home/01131/tg804093/anaconda2/lib/python2.7/site-packages/radical/utils/atfork/stdlib_fixer.py:65: UserWarning: logging handlers already registered.
  warnings.warn('logging handlers already registered.')


In [2]:
%%time
pilot_compute_service = PilotComputeService(coordination_url=COORDINATION_URL)

# create pilot job service and initiate a pilot job
pilot_compute_description = {
                         "service_url": 'subprocess://localhost',
                         #"service_url": 'ssh://localhost',
                         "number_of_processes": 24,                             
                         "working_directory": "/gpfs/flash/users/tg804093/bigjob/",
                        }

pilotjob = pilot_compute_service.create_pilot(pilot_compute_description=pilot_compute_description)
     
compute_data_service = ComputeDataServiceDecentral()
compute_data_service.add_pilot_compute_service(pilot_compute_service)

CPU times: user 50.3 ms, sys: 50 ms, total: 100 ms
Wall time: 2.93 s


In [5]:
%%time
# start work unit
compute_unit_description = {
        "executable": "/bin/date",
        "number_of_processes": 1,            
        "output": "stdout.txt",
        "error": "stderr.txt",
}    

for i in range(0,14):
    compute_unit = compute_data_service.submit_compute_unit_batch([compute_unit_description])
    
    
print("Finished setup. Waiting for scheduling of CU")
compute_data_service.wait()

Finished setup. Waiting for scheduling of CU
CPU times: user 246 ms, sys: 40.8 ms, total: 287 ms
Wall time: 4.28 s


In [ ]:
import time

scenarios = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384, 32768, 65536, 131072]
#scenarios = [16]    

for n in scenarios:
    start_time = time.time()
    cu_list = []
    for i in range(n):
        compute_unit_description = {
        "executable": "/bin/date",
        "number_of_processes": 1,            
        "output": "stdout.txt",
        "error": "stderr.txt",
        }    
        cu_list.append(compute_unit_description)

    compute_data_service.submit_compute_unit_batch(cu_list)
    compute_data_service.wait()
    end_time= time.time()
    print("BigJob-Batch, %d, Runtime, %.4f"%(n, (end_time-start_time)))

BigJob-Batch, 1, Runtime, 8.0396
BigJob-Batch, 2, Runtime, 6.0481
BigJob-Batch, 4, Runtime, 4.0706
BigJob-Batch, 8, Runtime, 6.1280
BigJob-Batch, 16, Runtime, 4.2941
BigJob-Batch, 32, Runtime, 6.4703


In [ ]:
pilotjob.cancel()